<a href="https://colab.research.google.com/github/joosk3R/jskRprac/blob/main/ligbm_%2B_catboost_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import lightgbm
from lightgbm import LGBMRegressor
import catboost
from catboost import CatBoostRegressor
import sklearn
from sklearn.model_selection import KFold
import IPython
from IPython.display import clear_output
import copy

In [ ]:
print('python:{}'.format('3.7.3'))
print('numpy:{}'.format(np.__version__))
print('pandas:{}'.format(pd.__version__))
print('lightgbm:{}'.format(lightgbm.__version__))
print('catboost:{}'.format(catboost.__version__))
print('sklearn:{}'.format(sklearn.__version__))
print('IPython:{}'.format(IPython.__version__))
print('copy:{}'.format('기본모듈'))
print('os:{}'.format('window10'))
print('cpu:{}'.format('i7-8750h'))
print('gpu:{}'.format('gtx1050ti'))
print('ram:{}'.format('ddr4 2667hz 8GB * 2'))

python:3.7.3
numpy:1.22.4
pandas:1.5.3
lightgbm:3.3.5
catboost:1.2
sklearn:1.2.2
IPython:7.34.0
copy:기본모듈
os:window10
cpu:i7-8750h
gpu:gtx1050ti
ram:ddr4 2667hz 8GB * 2


In [ ]:
FILE_PATH = '.\\data'

cat_mae_params = {
    'objective': 'MAE',
    'n_estimators': 10000,
    'early_stopping_rounds': 4,
} #catboost hyper parameter

lgbm_mae_params = {
    'objective': 'MAE',
    'boosting_type': 'goss',
    'n_estimators': 10000,
    'early_stopping_round': 15,
    'num_leaves':39,
} #lightgbm hyper parameter

In [ ]:
def CDH(xs): #cooling degree hour를 구현
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

def detect_outliers(df,ratio): #iqr 이상치제거
    outlier_indices = []
    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = ratio * IQR
    return ~(df < Q1 - outlier_step) | (df > Q3 + outlier_step)

In [ ]:
train_df = pd.read_csv(FILE_PATH+'\\train.csv', encoding = "cp949") #train_csv


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train =pd.read_csv("/content/drive/MyDrive/energe/real_train_2.csv")

In [ ]:
train.head()

,num,date_time,power,temp,wind,hum,prec,day,month,week,day_hour_mean,hour_mean,hour_std,sin_time,cos_time,THI,holiday,CDH
0,1,20220601 00,1085.28,18.6,0.9,42.0,0.0,2,6,22,1774.744615,1706.318118,446.882767,0.000000,1.000000,49.6576,0,-7.4
1,1,20220601 01,1047.36,18.0,1.1,45.0,0.0,2,6,22,1687.347692,1622.620235,439.662704,0.258819,0.965926,47.7625,0,-15.4
2,1,20220601 02,974.88,17.7,1.5,45.0,0.0,2,6,22,1571.483077,1506.971294,412.071906,0.500000,0.866025,47.2225,0,-23.7
3,1,20220601 03,953.76,16.7,1.4,48.0,0.0,2,6,22,1522.153846,1437.365647,391.205981,0.707107,0.707107,44.7856,0,-33.0
4,1,20220601 04,986.40,18.4,2.8,43.0,0.0,2,6,22,1506.793846,1447.321412,381.099697,0.866025,0.500000,49.0061,0,-40.6


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/energe/train.csv") #train_csv

In [ ]:
train_df['date_time'] = train["date_time"]

In [ ]:
train_df

In [ ]:
test =pd.read_csv("/content/drive/MyDrive/energe/real_test_2.csv")
test_df = pd.read_csv("/content/drive/MyDrive/energe/test.csv")


In [ ]:
test_df["date_time"] = test_df["num_date_time"].str[-11:]

In [ ]:
test_df["num"]=test['num']
train_df["num"]=train['num']

In [ ]:
test_df = test_df.drop(columns = "건물번호")


In [ ]:
train_df = train_df.drop(columns = "건물번호")

In [ ]:
train_df

,num_date_time,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),date_time,num
0,1_20220601 00,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,20220601 00,1
1,1_20220601 01,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,20220601 01,1
2,1_20220601 02,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88,20220601 02,1
3,1_20220601 03,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76,20220601 03,1
4,1_20220601 04,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40,20220601 04,1
...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04,20220824 19,100
203996,100_20220824 20,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96,20220824 20,100
203997,100_20220824 21,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12,20220824 21,100
203998,100_20220824 22,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08,20220824 22,100


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num_date_time  16800 non-null  object 
 1   일시             16800 non-null  object 
 2   기온(C)          16800 non-null  float64
 3   강수량(mm)        16800 non-null  float64
 4   풍속(m/s)        16800 non-null  float64
 5   습도(%)          16800 non-null  int64  
 6   num            16800 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 918.9+ KB


In [ ]:
test

,num,temp,wind,hum,prec,day,month,week,day_hour_mean,hour_mean,hour_std,sin_time,cos_time,THI,holiday,CDH
0,1,23.5,2.2,72,0.0,3,8,34,1627.800000,1706.318118,446.882767,0.000000,1.000000,58.3456,0,-2.5
1,1,23.0,0.9,72,0.0,3,8,34,1550.080000,1622.620235,439.662704,0.258819,0.965926,57.4456,0,-5.5
2,1,22.7,1.5,75,0.0,3,8,34,1431.120000,1506.971294,412.071906,0.500000,0.866025,57.8725,0,-8.8
3,1,22.1,1.3,78,0.0,3,8,34,1372.200000,1437.365647,391.205981,0.707107,0.707107,57.9376,0,-12.7
4,1,21.8,1.0,77,0.0,3,8,34,1381.720000,1447.321412,381.099697,0.866025,0.500000,56.9961,0,-16.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,22.5,0.9,84,0.0,2,8,35,964.873846,1010.462118,161.399578,-0.965926,0.258819,61.4824,0,-34.5
16796,100,20.7,0.4,95,0.0,2,8,35,882.184615,928.125176,137.566008,-0.866025,0.500000,65.2725,0,-34.4
16797,100,20.2,0.4,98,0.0,2,8,35,779.095385,830.032941,128.300189,-0.707107,0.707107,66.7056,0,-35.3
16798,100,20.1,1.1,97,0.0,2,8,35,663.267692,723.100235,112.464079,-0.500000,0.866025,65.7281,0,-36.8


In [ ]:
train_df = train_df.reindex([ '전력소비량(kWh)','num_date_time', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', 'date_time', 'num'] ,axis =1)

In [ ]:
train_df

,전력소비량(kWh),num_date_time,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),date_time,num,dayofyear,hour,weekday,hour_te,hour_te1,불쾌지수
0,1085.28,1_20220601 00,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,2022-06-01 00:00:00,1,152,0,2,0.000000e+00,1.000000,1
1,1047.36,1_20220601 01,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,2022-06-01 01:00:00,1,152,1,2,2.697968e-01,0.962917,1
2,974.88,1_20220601 02,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,2022-06-01 02:00:00,1,152,2,2,5.195840e-01,0.854419,1
3,953.76,1_20220601 03,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,2022-06-01 03:00:00,1,152,3,2,7.308360e-01,0.682553,1
4,986.40,1_20220601 04,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,2022-06-01 04:00:00,1,152,4,2,8.878852e-01,0.460065,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,881.04,100_20220824 19,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,2022-08-24 19:00:00,100,236,19,2,-8.878852e-01,0.460065,2
203996,798.96,100_20220824 20,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,2022-08-24 20:00:00,100,236,20,2,-7.308360e-01,0.682553,2
203997,825.12,100_20220824 21,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,2022-08-24 21:00:00,100,236,21,2,-5.195840e-01,0.854419,2
203998,640.08,100_20220824 22,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,2022-08-24 22:00:00,100,236,22,2,-2.697968e-01,0.962917,2


In [ ]:

train_df['date_time'] = pd.to_datetime(train_df['date_time'])
train_df['dayofyear'] = train_df['date_time'].dt.dayofyear
train_df['hour'] = train_df['date_time'].dt.hour
train_df['weekday'] = train_df['date_time'].dt.weekday #time feature

train_df['hour_te'] = np.sin(2*np.pi*(train_df['hour'])/23)
train_df['hour_te1'] = np.cos(2*np.pi*(train_df['hour'])/23) #time encoding hour

t = 9/5*train_df['기온(C)']
train_df['불쾌지수'] = t - 0.55*(1-train_df['습도(%)']/100)*(t-26)+32
train_df['불쾌지수'] = pd.cut(train_df['불쾌지수'], bins = [0, 68, 75, 80, 200], labels = [1,2,3,4]) #불쾌지수는 카테고리로 나누는게 성능상승에 도움이 됨

train_dfs  = []
for i in range(1,101,1):
    train_dfs.append(train_df[train_df['num']==i])


for i in range(len(train_dfs)):
    train_dfs[i] = train_dfs[i].drop(columns=['풍속(m/s)','강수량(mm)','일조(hr)','num','date_time',"일사(MJ/m2)",'num_date_time',"일시"]) #쓸모없는 특징 drop


test_df['date_time'] = pd.to_datetime(test_df['date_time'])


test_df['dayofyear'] = test_df['date_time'].dt.dayofyear
test_df['hour'] = test_df['date_time'].dt.hour
test_df['weekday'] = test_df['date_time'].dt.weekday #time feature

test_df['hour_te'] = np.sin(2*np.pi*(test_df['hour'])/23)
test_df['hour_te1'] = np.cos(2*np.pi*(test_df['hour'])/23) #time encoding hour

t = 9/5*test_df['기온(C)']
test_df['불쾌지수'] = t - 0.55*(1-test_df['습도(%)']/100)*(t-26)+32
test_df['불쾌지수'] = pd.cut(test_df['불쾌지수'], bins = [0, 68, 75, 80, 200], labels = [1,2,3,4]) #불쾌지수는 카테고리로 나누는게 성능상승에 도움이 됨

test_dfs  = []
for i in range(1,101,1):
    test_dfs.append(test_df[test_df['num']==i])

for i in range(len(test_dfs)):
    test_dfs[i] = test_dfs[i].drop(columns=['풍속(m/s)','강수량(mm)','num','date_time','num_date_time',"일시"]) #쓸모없는 특징 drop


for i in range(100): #cdh 특징 추가
    train_dfs[i]['cdh'] = CDH(np.concatenate([train_dfs[i]['기온(C)'].values,test_dfs[i]['기온(C)'].values]))[:-len(test_dfs[i])]
    test_dfs[i]['cdh'] = CDH(np.concatenate([train_dfs[i]['기온(C)'].values,test_dfs[i]['기온(C)'].values]))[-len(test_dfs[i]):]




In [ ]:
train_dfs[0]

,전력소비량(kWh),기온(C),습도(%),dayofyear,hour,weekday,hour_te,hour_te1,불쾌지수,cdh
0,1085.28,18.6,42.0,152,0,2,0.000000e+00,1.000000,1,-7.4
1,1047.36,18.0,45.0,152,1,2,2.697968e-01,0.962917,1,-15.4
2,974.88,17.7,45.0,152,2,2,5.195840e-01,0.854419,1,-23.7
3,953.76,16.7,48.0,152,3,2,7.308360e-01,0.682553,1,-33.0
4,986.40,18.4,43.0,152,4,2,8.878852e-01,0.460065,1,-40.6
...,...,...,...,...,...,...,...,...,...,...
2035,2517.12,25.5,63.0,236,19,2,-8.878852e-01,0.460065,2,3.1
2036,2208.96,25.0,66.0,236,20,2,-7.308360e-01,0.682553,2,4.3
2037,1806.24,24.6,67.0,236,21,2,-5.195840e-01,0.854419,2,4.2
2038,1387.20,24.1,70.0,236,22,2,-2.697968e-01,0.962917,2,2.3


In [ ]:
#train_x와 train_y로 나눔
train_x = []
train_y = []
for i in range(len(train_dfs)):
    train_x.append(copy.deepcopy(train_dfs[i][train_dfs[i].columns[1:]]))
    train_y.append(copy.deepcopy(train_dfs[i][train_dfs[i].columns[0]]))


In [ ]:
train_dfs

In [ ]:
train_x

In [ ]:
train_y

In [ ]:
#이상치 제거 iqr은 1.25
for i in range(100):
    idx = detect_outliers(train_y[i],1.25)
    train_y[i] = train_y[i][idx]
    train_x[i] = train_x[i][idx]

In [ ]:
#과적합 방지를 위해 여러 k_fold로 반복하도록 설정
#특징중 몇개를 뺄경우 성능 향상을 기대할수 있고 과적합 또한 방지 가능하다
random_seed = 0
dcs = [[],['기온(C)'], ['습도(%)'], ['hour_te','hour_te1'], ['불쾌지수'], ['cdh']]
ks = [3,4,5]

In [ ]:
train_dfs[2]

,전력소비량(kWh),기온(C),습도(%),dayofyear,hour,weekday,hour_te,hour_te1,불쾌지수,cdh
4080,926.28,18.6,42.0,152,0,2,0.000000e+00,1.000000,1,-7.4
4081,884.52,18.0,45.0,152,1,2,2.697968e-01,0.962917,1,-15.4
4082,882.36,17.7,45.0,152,2,2,5.195840e-01,0.854419,1,-23.7
4083,867.24,16.7,48.0,152,3,2,7.308360e-01,0.682553,1,-33.0
4084,880.92,18.4,43.0,152,4,2,8.878852e-01,0.460065,1,-40.6
...,...,...,...,...,...,...,...,...,...,...
6115,2729.16,25.5,63.0,236,19,2,-8.878852e-01,0.460065,2,3.1
6116,2664.00,25.0,66.0,236,20,2,-7.308360e-01,0.682553,2,4.3
6117,2135.16,24.6,67.0,236,21,2,-5.195840e-01,0.854419,2,4.2
6118,1158.84,24.1,70.0,236,22,2,-2.697968e-01,0.962917,2,2.3


In [ ]:
test_dfs[0]

,기온(C),습도(%),dayofyear,hour,weekday,hour_te,hour_te1,불쾌지수,cdh
0,23.5,72,237,0,3,0.000000e+00,1.000000,2,-4.8
1,23.0,72,237,1,3,2.697968e-01,0.962917,2,-9.4
2,22.7,75,237,2,3,5.195840e-01,0.854419,2,-13.8
3,22.1,78,237,3,3,7.308360e-01,0.682553,2,-18.3
4,21.8,77,237,4,3,8.878852e-01,0.460065,2,-22.8
...,...,...,...,...,...,...,...,...,...
163,23.5,75,243,19,2,-8.878852e-01,0.460065,2,-36.5
164,22.4,80,243,20,2,-7.308360e-01,0.682553,2,-34.0
165,21.7,81,243,21,2,-5.195840e-01,0.854419,2,-32.6
166,21.1,83,243,22,2,-2.697968e-01,0.962917,2,-33.0


In [ ]:
answer_df = pd.read_csv("/content/drive/MyDrive/energe/sample_submission.csv")

for dc in dcs:#d특정 feature dc를 drop 시킴
    for k in ks:#kfold 의 nspilt 의 값 k
        folds = []
        for i in range(len(train_dfs)):
            cross=KFold(n_splits=k,shuffle=True,random_state=random_seed)
            fold=[]
            for train_idx, valid_idx in cross.split(train_x[i], train_y[i]):
                fold.append((train_idx, valid_idx))
            folds.append(fold)

        for i in range(len(train_dfs)):
            for fold in range(k):
                print(dc,random_seed,k,i)
                train_idx, valid_idx = folds[i][fold]
                X_train=np.array(train_x[i].drop(columns=dc).iloc[train_idx])
                y_train=np.array(train_y[i].iloc[train_idx])
                X_valid=np.array(train_x[i].drop(columns=dc).iloc[valid_idx])
                y_valid=np.array(train_y[i].iloc[valid_idx])
                #catboost 학습
                model=CatBoostRegressor(**cat_mae_params)
                model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=100)
                v = model.predict(np.array(test_dfs[i][train_x[i].drop(columns=dc).columns])) * 0.3
                #lgbm 학습
                model=LGBMRegressor(**lgbm_mae_params)
                model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=100)
                v += model.predict(np.array(test_dfs[i][train_x[i].drop(columns=dc).columns])) * 0.7

                answer_df['answer'].iloc[(i)*168:(i+1)*168] += v/(len(ks)*k*len(dcs))
                clear_output(True)

        random_seed += 1
answer_df.to_csv('/content/drive/MyDrive/energe/answer_df.csv', index=False) #파일 저장

['기온(C)'] 5 5 77
0:	learn: 634.4693048	test: 652.5564008	best: 652.5564008 (0)	total: 3.89ms	remaining: 38.9s
100:	learn: 86.2593595	test: 83.7634990	best: 83.7634990 (100)	total: 92.8ms	remaining: 9.1s
200:	learn: 62.4498540	test: 60.2846247	best: 60.2846247 (200)	total: 162ms	remaining: 7.89s
